In [ ]:
from collections import defaultdict
from pprint import pprint
import time
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from balance_date import BalanceDate

import utils
pd.options.display.max_rows = 10 
%matplotlib inline

from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.model_selection import train_test_split
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)  
from sklearn.externals import joblib  
from sklearn.feature_selection import RFE,RFECV

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import time


from tqdm import tqdm, trange
with open("./config.json") as f:
    config = json.loads(f.read())

In [ ]:
FEATURE = BalanceDate.get_feature() 
OUT_PATH = os.path.join(".", "out", time.strftime("%Y%m%d")) 
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
print("feature：", FEATURE)
print("out path：", OUT_PATH)

# read data

In [ ]:
cv_kind = 1
train_set_kind = 1

X_test1, y_test1, X_test2, y_test2 = BalanceDate.get_test_values()  
X_train, y_train, g_train = BalanceDate.get_train_values(train_set_kind)  
cvs = BalanceDate.split_cv(kind=cv_kind, train_set_kind=train_set_kind)
cvs_method = BalanceDate.split_cv(kind=cv_kind, train_set_kind=train_set_kind, return_kind=2)

print("test1")
utils.solubility_distribute(y_test1, 0, 1)
print("test2")
utils.solubility_distribute(y_test2, 0, 1)
print("train")
utils.solubility_distribute(y_train, 0, 1)

# read model

In [ ]:
names = ['RandomForeast', 'lightGBM', 'XGBoost']

models = [
    RandomForestClassifier(random_state=0),
    lgb.LGBMClassifier(random_state=0),  # gbm     
    xgb.XGBClassifier(random_state=0),
]

print(names)

# train

In [ ]:
res_cv = []  
res_blind = []  
for i in range(len(names)):
    name = names[i]
    model = models[i]
    print(name, model)
    print("10cv")
    stime = time.time()
    res_cv.append( utils.CVUtil(model, name, cvs_method).set_data(X_train, y_train, g_train).fit())
    etime = time.time()
    print("time:", etime - stime)
    print("blind")
    res_blind.append( utils.BlindTestUtil(model, name).set_data(X_train, y_train, X_test1, y_test1, X_test2, y_test2).fit() )

# save result

In [ ]:
utils.result_output(res_cv, res_blind, names, OUT_PATH, "10cv_3class", False)  
utils.result_output(res_cv, res_blind, names, OUT_PATH, "10cv_3class", True)  